# Notebook contains training and testing of different classifiers


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_excel('edited master copy.xlsx')
dataset = dataset[dataset['sex_upon_intake'].notna()]


##taking intake age and intake number
X = dataset.iloc[:, [1,4,5,6,7,9,11,13]].values
y = dataset.iloc[:, 14].values


# Feature Scaling
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)








In [2]:


len(dataset)

47802

In [0]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.compose import ColumnTransformer
labelcoder_y = LabelEncoder()
y = labelcoder_y.fit_transform(y)








In [4]:
y

array([6, 6, 6, ..., 6, 0, 3])

In [5]:
le_name_mapping = dict(zip(labelcoder_y.classes_, labelcoder_y.transform(labelcoder_y.classes_)))
print(le_name_mapping)

{'Adoption': 0, 'Died': 1, 'Disposal': 2, 'Euthanasia': 3, 'Missing': 4, 'Relocate': 5, 'Return to Owner': 6, 'Rto-Adopt': 7, 'Transfer': 8}


In [6]:
y[0:10]

array([6, 6, 6, 4, 6, 6, 6, 6, 6, 6])

In [7]:
X[100, :]


array(['Dog', 'Aged', 'Stray', 'Spayed Female', 14.0, 1, 'Spayed Female',
       5112.231944], dtype=object)

In [0]:
ct = ColumnTransformer([("encoding", OneHotEncoder(), [0,1,2,3,6])], remainder = 'passthrough')
X = ct.fit_transform(X)

In [9]:
X[100, :]

<1x30 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [0]:
# Splitting the dataset into the Training set and Test set
#from sklearn.cross_validation import train_test_split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [11]:
X_train

<38241x30 sparse matrix of type '<class 'numpy.float64'>'
	with 305750 stored elements in Compressed Sparse Row format>

In [12]:
y_train

array([5, 0, 1, ..., 0, 7, 0])

In [13]:
from xgboost import XGBClassifier
classifier1 = XGBClassifier()
classifier1.fit(X_train, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
y_pred = classifier1.predict(X_test)

In [15]:
from sklearn.metrics import confusion_matrix
cm_xgboost = confusion_matrix(y_test, y_pred)
cm_xgboost


array([[2203,    0,    0,   17,    0,    0,  312,    0,   75],
       [ 318,    0,    0,   28,    0,    0,   66,    0,  170],
       [   0,    0,    7,   26,    0,    0,    1,    0,    0],
       [  62,    1,    1,  516,    0,    0,   82,    0,  127],
       [ 348,    1,    0,    8,    0,    0,   67,    0,  133],
       [ 298,    0,    0,    7,    0,    0,   80,    0,  139],
       [ 271,    0,    0,   25,    0,    0, 1588,    0,   44],
       [ 282,    0,    0,    7,    0,    0,   64,    0,  120],
       [ 612,    1,    0,   51,    0,    0,  236,    0, 1167]])

{'Adoption': 0, 'Died': 1, 'Disposal': 2, 'Euthanasia': 3, 'Missing': 4, 'Relocate': 5, 'Return to Owner': 6, 'Rto-Adopt': 7, 'Transfer': 8}

model performs well on classes 'Adoption' and 'transfer' as most of the samples are distributed between these two classes. Also poor performane is observed for 'disposal' and 'RTO-Adopt'.

In [16]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier1, X = X_train, y = y_train, cv = 5)
accuracies.mean()


0.5719776589058323

# So a Xgboost without any parameter tuning gives us an average accuracy = 0.57 .

Lets implement some other well known classfiers just to see how well they are performing
 

In [19]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 2.9MB/s 


I will use a a binary relevance model from skmultilearn

In this case an ensemble of single-label binary classifiers is trained, one for each class. Each classifier predicts either the membership or the non-membership of one class. The union of all classes that were predicted is taken as the multi-label output. This approach is popular because it is easy to implement, however it also ignores the possible correlations between class labels.

In [61]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
ndarray = np.ndarray(shape=(9561,1),dtype = int )
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier2 = BinaryRelevance(GaussianNB())
# train
classifier2.fit(X_train, y_train)
# predict
predictions = classifier2.predict(X_test)
predictions.toarray(out=ndarray)
# accuracy
#print("Accuracy = ",accuracy_score(y_test,predictions))

array([[3],
       [0],
       [8],
       ...,
       [6],
       [0],
       [6]])

In [62]:
y_test

array([8, 0, 8, ..., 4, 6, 7])

In [63]:
ndarray[: , 0]

array([3, 0, 8, ..., 6, 0, 6])

In [64]:
print("Accuracy = ",accuracy_score(y_test,ndarray[: , 0]))

Accuracy =  0.489070180943416


In [65]:
from sklearn.metrics import confusion_matrix
cm_binaryrel = confusion_matrix(y_test, ndarray[: , 0])
cm_binaryrel


array([[2031,   11,   13,   48,    1,    7,  398,    3,   95],
       [ 301,    3,   11,   39,    0,    2,   82,    1,  143],
       [   0,    0,   31,    2,    0,    0,    1,    0,    0],
       [  60,    1,  340,  221,    0,    3,   85,    0,   79],
       [ 339,    1,    1,   26,    0,    1,   77,    0,  112],
       [ 281,    0,    4,   20,    0,    4,   95,    3,  117],
       [ 397,    2,    4,   42,    1,    5, 1423,    0,   54],
       [ 264,    1,    2,   13,    0,    1,   80,    0,  112],
       [ 602,    3,   18,  157,    0,    8,  314,    2,  963]])

Though there is a decrease in accuracy , f1 score on test dataset is likely to increase . if we compare both the cm you will observe that prediction for class 'Disposal' has now increased.

# Classfier chains

A chain of binary classifiers C0, C1, . . . , Cn is constructed, where a classifier Ci uses the predictions of all the classifier Cj , where j < i. This way the method, also called classifier chains (CC), can take into account label correlations.

In [59]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
# initialize classifier chains multi-label classifier
classifier3 = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier3.fit(X_train, y_train)
# predict
predictions = classifier3.predict(X_test)
ndarray = np.ndarray(shape=(9561,1),dtype = float)
predictions.toarray(out=ndarray)
print("Accuracy = ",accuracy_score(y_test,ndarray[: , 0]))

Accuracy =  0.4453509047170798


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [57]:
predictions

<9561x1 sparse matrix of type '<class 'numpy.float64'>'
	with 6918 stored elements in Compressed Sparse Column format>

In [72]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier4 = LabelPowerset(LogisticRegression())
# train
classifier4.fit(X_train, y_train)
# predict
predictions4 = classifier.predict(X_test)
# accuracy
ndarray = np.ndarray(shape=(9561,1),dtype = int)
predictions4.toarray(out=ndarray)
print("Accuracy = ",accuracy_score(y_test,ndarray[: , 0]))




Accuracy =  0.489070180943416


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [73]:
from sklearn.metrics import confusion_matrix
cm_labelpower = confusion_matrix(y_test, ndarray[: , 0])
cm_labelpower

array([[2031,   11,   13,   48,    1,    7,  398,    3,   95],
       [ 301,    3,   11,   39,    0,    2,   82,    1,  143],
       [   0,    0,   31,    2,    0,    0,    1,    0,    0],
       [  60,    1,  340,  221,    0,    3,   85,    0,   79],
       [ 339,    1,    1,   26,    0,    1,   77,    0,  112],
       [ 281,    0,    4,   20,    0,    4,   95,    3,  117],
       [ 397,    2,    4,   42,    1,    5, 1423,    0,   54],
       [ 264,    1,    2,   13,    0,    1,   80,    0,  112],
       [ 602,    3,   18,  157,    0,    8,  314,    2,  963]])

# Prediction and output file generation

We will be using binary relevance classifier to predict outcome type 


In [0]:
test_dataset = pd.read_excel('edited master test.xlsx')
X_test1 = test_dataset.iloc[:, [1,4,5,6,7,9,11,13]].values

In [75]:
len(test_dataset)

31869

In [76]:
X_test1[2,:]

array(['Cat', 'Nursing', 'Stray', 'Intact Male', 0.076712329, 1,
       'Neutered Male', 60.52013889], dtype=object)

In [0]:
ct_test = ColumnTransformer([("encoding", OneHotEncoder(), [0,1,2,3,6])], remainder = 'passthrough')
X_test1 = ct_test.fit_transform(X_test1)

In [78]:
X_test1[2,:]

<1x30 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [0]:
y_pred1 = classifier2.predict(X_test1)


In [81]:
y_pred1

<31869x1 sparse matrix of type '<class 'numpy.int64'>'
	with 16384 stored elements in Compressed Sparse Column format>

In [82]:
ndarray = np.ndarray(shape=(31869,1),dtype = int)
y_pred1.toarray(out=ndarray)

array([[8],
       [8],
       [8],
       ...,
       [2],
       [6],
       [6]])

In [0]:
y_pred1=ndarray[: , 0]

In [85]:
y_pred1

array([8, 8, 8, ..., 2, 6, 6])

In [0]:
y_pred1 = labelcoder_y.inverse_transform(y_pred1)

In [87]:
y_pred1

array(['Transfer', 'Transfer', 'Transfer', ..., 'Disposal',
       'Return to Owner', 'Return to Owner'], dtype=object)

In [0]:
output_file = test_dataset

In [0]:
output_file['outcome_type'] = y_pred1 

In [90]:
output_file['outcome_type']

0               Transfer
1               Transfer
2               Transfer
3               Transfer
4               Transfer
              ...       
31864           Transfer
31865           Disposal
31866           Disposal
31867    Return to Owner
31868    Return to Owner
Name: outcome_type, Length: 31869, dtype: object

In [0]:
#output_file

In [0]:
output_file_final = output_file[['animal_id_outcome','outcome_type']]

In [0]:
output_file_final.to_csv('output_file_binaryrel.csv',index=False)